In [28]:
import pymysql
import pandas as pd
import re
import itertools

In [15]:
# 单品种获取
code = 'czce_ap'
conn = pymysql.connect(host = "localhost",port=3306,user='root',passwd='0000',db='db_eastmoney',charset="utf8")
sql_query = 'SELECT * FROM '+code
# 从mysql中读取表
df1 = pd.read_sql(sql_query, con=conn)
# 中文转码
df1.name = df1.name.apply(lambda x:eval(repr(x).replace('\\\\', '\\')))
# 去掉主力和次主力的行 带有- 的行
df1 = df1[(df1.name.str.contains('主力')==False) & (df1.new_p!='-')]
# TA特殊处理
conn.close()  #使用完后记得关掉
name = ''.join(re.findall(u"[\u4e00-\u9fa5]+", df1.loc[0,'name']))
codelist = df1.code.tolist()
df1,name,codelist

(    code      excode   name   new_p    open    high     low  pre_close  \
 0  AP110  CZCE|AP110  苹果110  6160.0  6180.0  6195.0  6141.0     6199.0   
 1  AP111  CZCE|AP111  苹果111  5996.0  6010.0  6022.0  5985.0     6029.0   
 2  AP112  CZCE|AP112  苹果112  6045.0  6049.0  6065.0  6030.0     6078.0   
 3  AP201  CZCE|AP201  苹果201  6158.0  6180.0  6191.0  6151.0     6212.0   
 4  AP203  CZCE|AP203  苹果203  6120.0  6141.0  6154.0  6119.0     6163.0   
 5  AP204  CZCE|AP204  苹果204  6249.0  6227.0  6263.0  6225.0     6265.0   
 6  AP205  CZCE|AP205  苹果205  6406.0  6400.0  6430.0  6382.0     6411.0   
 
    volume  dominant  
 0  171216         1  
 1    1195         0  
 2     395         0  
 3   18572         0  
 4     471         0  
 5     143         0  
 6    1505         0  ,
 '苹果',
 ['AP110', 'AP111', 'AP112', 'AP201', 'AP203', 'AP204', 'AP205'])

In [29]:
# 多品种获取
import pymysql.cursors 
 
 
dbname = 'db_eastmoney'
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='0000',
                             db=dbname,
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor)
results =[]
try:
    with connection.cursor() as cursor:
        sql = '''SHOW TABLES'''
        cursor.execute(sql)
        result = cursor.fetchall() 
        for i in range(len(result)):
            results.append(result[i]['Tables_in_'+dbname])
finally:
    connection.close()

print(len(results),results)

60 ['czce_ap', 'czce_cf', 'czce_cj', 'czce_cy', 'czce_fg', 'czce_jr', 'czce_lr', 'czce_ma', 'czce_oi', 'czce_pf', 'czce_pk', 'czce_pm', 'czce_ri', 'czce_rm', 'czce_rs', 'czce_sa', 'czce_sf', 'czce_sm', 'czce_sr', 'czce_ta', 'czce_ur', 'czce_wh', 'czce_zc', 'dce_a', 'dce_b', 'dce_bb', 'dce_c', 'dce_cs', 'dce_eb', 'dce_eg', 'dce_fb', 'dce_i', 'dce_j', 'dce_jd', 'dce_jm', 'dce_l', 'dce_lh', 'dce_m', 'dce_p', 'dce_pg', 'dce_pp', 'dce_rr', 'dce_v', 'dce_y', 'shfe_ag', 'shfe_al', 'shfe_au', 'shfe_bu', 'shfe_cu', 'shfe_fu', 'shfe_hc', 'shfe_ni', 'shfe_pb', 'shfe_rb', 'shfe_ru', 'shfe_sn', 'shfe_sp', 'shfe_ss', 'shfe_wr', 'shfe_zn']


In [30]:
abitrage_dic = {}
conn = pymysql.connect(host = "localhost",port=3306,user='root',passwd='0000',db='db_eastmoney',charset="utf8")
for i in results:
    sql_query = 'SELECT * FROM '+i
    # 从mysql中读取表
    df1 = pd.read_sql(sql_query, con=conn)
    # 中文转码
    df1.name = df1.name.apply(lambda x:eval(repr(x).replace('\\\\', '\\')))
    # TA特殊处理
    if 'TA' in df1.loc[0,'name']:
        # 提取字母
        name = ''.join(re.findall(r'[A-Za-z]', df1.loc[0,'name']))  
    else:
        # 提取中文
        name = ''.join(re.findall(u"[\u4e00-\u9fa5]+", df1.loc[0,'name']))
        
    # 提取字母  code  AP 
    code = ''.join(re.findall(r'[A-Za-z]', df1.loc[0,'code']))
    # 去掉主力和次主力的行 带有- 的行
    df1 = df1[(df1.name.str.contains('主力')==False) & (df1.new_p!='-')]
    # 转换类型
    df1['new_p'] = df1['new_p'].astype('float64')
    df1['volume'] = df1['volume'].astype('int64')
    df1['money'] = df1['new_p']*df1['volume']
    
    if len(df1)>=2:
        print(name,df1.code.tolist())
conn.close()  #使用完后记得关掉

苹果 ['AP110', 'AP111', 'AP112', 'AP201', 'AP203', 'AP204', 'AP205']
一号棉花 ['CF107', 'CF109', 'CF111', 'CF201', 'CF203', 'CF205']
红枣 ['CJ109', 'CJ112', 'CJ201', 'CJ203', 'CJ205']
棉纱 ['CY109', 'CY201']
玻璃 ['FG107', 'FG108', 'FG109', 'FG110', 'FG111', 'FG112', 'FG201', 'FG202', 'FG203', 'FG204', 'FG205', 'FG206']
甲醇 ['MA108', 'MA109', 'MA110', 'MA111', 'MA112', 'MA201', 'MA202', 'MA203', 'MA204', 'MA205']
菜油 ['OI107', 'OI109', 'OI111', 'OI201', 'OI203', 'OI205']
短纤 ['PF107', 'PF109', 'PF110', 'PF111', 'PF112', 'PF201', 'PF202', 'PF205', 'PF206']
花生 ['PK110', 'PK111', 'PK112', 'PK201', 'PK204']
普麦 ['PM107', 'PM109']
菜粕 ['RM107', 'RM108', 'RM109', 'RM111', 'RM201', 'RM203', 'RM205']
纯碱 ['SA107', 'SA108', 'SA109', 'SA110', 'SA111', 'SA112', 'SA201', 'SA202', 'SA203', 'SA204', 'SA205', 'SA206']
硅铁 ['SF107', 'SF108', 'SF109', 'SF110', 'SF111', 'SF112', 'SF201', 'SF202', 'SF203', 'SF204', 'SF205', 'SF206']
锰硅 ['SM107', 'SM108', 'SM109', 'SM110', 'SM111', 'SM112', 'SM201', 'SM202', 'SM205']
白糖 ['S

In [40]:
# aksahre的数据
# 返回值分别为品种、现货价格、最近交割合约、最近交割合约价格、主力合约、主力合约价格、
# 最近合约基差值、主力合约基差值、最近合约基差率、主力合约基差率.
import akshare as ak
df = ak.futures_spot_price("20210709")

In [43]:
import akshare as ak
futures_spot_price_previous_df = ak.futures_spot_price_previous('20210709')
futures_spot_price_previous_df.head()

,商品,现货价格,主力合约代码,主力合约价格,主力合约基差,主力合约变动百分比,180日内主力基差最高,180日内主力基差最低,180日内主力基差平均
0,铜,68700,2108,68630,70.00,0.10,1160.00,-740.00,-39.02
1,螺纹钢,4941.78,2110,5356,-414.00,-8.38,100.00,-438.22,-126.76
2,锌,23020,2108,22290,730.00,3.17,910.00,215.00,605.97
3,铝,18860,2108,18800,60.00,0.32,530.00,-325.00,10.08
4,黄金,376.95,2112,378.36,-1.41,-0.37,2.94,-5.44,-1.52
